# Lesson A1: Lifetimes & Advanced Memory Management

**Duration**: 150-180 minutes  
**Stage**: Advanced (Mastery)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Understand and apply explicit lifetime annotations in complex scenarios
2. Design APIs with lifetime parameters for zero-cost abstractions
3. Resolve advanced lifetime conflicts and borrow checker issues
4. Implement lifetime-aware data structures and iterators
5. Apply lifetime elision rules and understand when explicit annotations are needed

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. What are the three ownership rules in Rust?
2. What's the difference between `&T` and `&mut T`?
3. How do you handle errors using custom error types?
4. What are the benefits of Rust's ownership system?

**Answers**: 1) Each value has one owner, ownership can be transferred, owner is responsible for cleanup, 2) Immutable vs mutable references, 3) Define enum with variants, implement Display and Error traits, 4) Memory safety without garbage collection, zero-cost abstractions, thread safety

---

## 🧠 Key Concepts

### What are Lifetimes?

**Lifetimes** are Rust's way of ensuring that references are valid for as long as needed:
- **Scope-based**: Every reference has a lifetime (scope where it's valid)
- **Compile-time**: Lifetime checking happens at compile time, zero runtime cost
- **Relationship Tracking**: Lifetimes track relationships between references
- **Memory Safety**: Prevent dangling pointers and use-after-free bugs

### Lifetime Annotation Syntax

- **Generic Lifetime Parameters**: `'a`, `'b`, `'static`
- **Function Signatures**: `fn foo<'a>(x: &'a str) -> &'a str`
- **Struct Definitions**: `struct Foo<'a> { field: &'a str }`
- **Implementation Blocks**: `impl<'a> Foo<'a> { ... }`

### Lifetime Elision Rules

Rust can often infer lifetimes automatically:
1. **Input lifetimes**: Each parameter gets its own lifetime
2. **Single input**: Output lifetime matches input lifetime
3. **Method with `&self`**: Output lifetime matches `&self`

### Advanced Lifetime Patterns

- **Multiple lifetimes**: Different references with different scopes
- **Lifetime bounds**: `T: 'a` means T must live at least as long as 'a
- **Higher-ranked trait bounds**: `for<'a>` syntax for flexible APIs
- **Static lifetime**: `'static` for data that lives for entire program

---

## 🔬 Live Code Exploration

### Basic Lifetime Annotations

In [ ]:
// Basic lifetime examples

// Function that returns the longer of two string slices
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

// Function with multiple lifetime parameters
fn first_word<'a, 'b>(x: &'a str, _y: &'b str) -> &'a str {
    x.split_whitespace().next().unwrap_or("")
}

// Struct with lifetime parameter
#[derive(Debug)]
struct ImportantExcerpt<'a> {
    part: &'a str,
}

impl<'a> ImportantExcerpt<'a> {
    // Method with lifetime elision
    fn level(&self) -> i32 {
        3
    }
    
    // Method that returns a reference with same lifetime as self
    fn announce_and_return_part(&self, announcement: &str) -> &str {
        println!("Attention please: {}", announcement);
        self.part
    }
}

fn basic_lifetimes_demo() {
    println!("=== Basic Lifetimes Demo ===");
    
    // Simple lifetime example
    let string1 = String::from("abcd");
    let string2 = "xyz";
    
    let result = longest(string1.as_str(), string2);
    println!("The longest string is: '{}'", result);
    
    // Multiple lifetimes
    let sentence = "Hello world from Rust";
    let other = "Goodbye";
    let first = first_word(sentence, other);
    println!("First word: '{}'", first);
    
    // Struct with lifetime
    let novel = String::from("Call me Ishmael. Some years ago...");
    let first_sentence = novel.split('.').next().expect("Could not find a '.'");
    let excerpt = ImportantExcerpt {
        part: first_sentence,
    };
    
    println!("Excerpt: {:?}", excerpt);
    println!("Level: {}", excerpt.level());
    
    let announcement = "Today we're reading a classic";
    let part = excerpt.announce_and_return_part(announcement);
    println!("Returned part: '{}'", part);
}

basic_lifetimes_demo();

### Advanced Lifetime Patterns

In [ ]:
use std::fmt::Display;

// Advanced lifetime patterns

// Function with lifetime bounds
fn longest_with_an_announcement<'a, T>(
    x: &'a str,
    y: &'a str,
    ann: T,
) -> &'a str
where
    T: Display,
{
    println!("Announcement! {}", ann);
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

// Struct with multiple lifetime parameters
#[derive(Debug)]
struct Context<'s, 'c> {
    string_data: &'s str,
    config_data: &'c str,
}

impl<'s, 'c> Context<'s, 'c> {
    fn new(string_data: &'s str, config_data: &'c str) -> Self {
        Context {
            string_data,
            config_data,
        }
    }
    
    // Method that returns reference with specific lifetime
    fn get_string_data(&self) -> &'s str {
        self.string_data
    }
    
    fn get_config_data(&self) -> &'c str {
        self.config_data
    }
    
    // Method that combines data with different lifetimes
    fn format_info(&self) -> String {
        format!("Data: '{}', Config: '{}'", self.string_data, self.config_data)
    }
}

// Function that works with any lifetime
fn process_data<'a>(data: &'a str) -> (&'a str, usize) {
    let trimmed = data.trim();
    (trimmed, trimmed.len())
}

// Static lifetime example
static GLOBAL_CONFIG: &str = "production";

fn get_config() -> &'static str {
    GLOBAL_CONFIG
}

// Function that demonstrates lifetime subtyping
fn choose_str<'a, 'b: 'a>(first: &'a str, second: &'b str, use_first: bool) -> &'a str {
    if use_first {
        first
    } else {
        second  // 'b: 'a means 'b outlives 'a, so this is safe
    }
}

fn advanced_lifetimes_demo() {
    println!("\n=== Advanced Lifetimes Demo ===");
    
    // Lifetime bounds example
    let string1 = String::from("long string is long");
    let string2 = String::from("short");
    let announcement = "Comparing strings";
    
    let result = longest_with_an_announcement(
        string1.as_str(),
        string2.as_str(),
        announcement,
    );
    println!("Longest: '{}'", result);
    
    // Multiple lifetime parameters
    let data = String::from("  important data  ");
    let config = "debug";
    
    let context = Context::new(data.as_str(), config);
    println!("Context: {:?}", context);
    println!("Formatted: {}", context.format_info());
    
    // Processing with lifetimes
    let raw_data = "  Hello, Rust!  ";
    let (processed, length) = process_data(raw_data);
    println!("Processed '{}' (length: {})", processed, length);
    
    // Static lifetime
    let config = get_config();
    println!("Global config: {}", config);
    
    // Lifetime subtyping
    let long_lived = String::from("I live long");
    {
        let short_lived = String::from("I'm temporary");
        let chosen = choose_str(long_lived.as_str(), short_lived.as_str(), false);
        println!("Chosen: '{}'", chosen);
    }
    // short_lived is dropped here, but that's okay because we used it within its scope
}

advanced_lifetimes_demo();

### Lifetime-Aware Data Structures

In [ ]:
// Complex data structures with lifetimes

// Parser that holds references to input data
#[derive(Debug)]
struct Parser<'a> {
    input: &'a str,
    position: usize,
}

impl<'a> Parser<'a> {
    fn new(input: &'a str) -> Self {
        Parser { input, position: 0 }
    }
    
    fn current_char(&self) -> Option<char> {
        self.input.chars().nth(self.position)
    }
    
    fn advance(&mut self) {
        if self.position < self.input.len() {
            self.position += 1;
        }
    }
    
    fn parse_word(&mut self) -> Option<&'a str> {
        let start = self.position;
        
        // Skip whitespace
        while let Some(ch) = self.current_char() {
            if !ch.is_whitespace() {
                break;
            }
            self.advance();
        }
        
        let word_start = self.position;
        
        // Collect word characters
        while let Some(ch) = self.current_char() {
            if ch.is_whitespace() {
                break;
            }
            self.advance();
        }
        
        if word_start < self.position {
            Some(&self.input[word_start..self.position])
        } else {
            None
        }
    }
    
    fn remaining(&self) -> &'a str {
        &self.input[self.position..]
    }
}

// Iterator that yields string slices
struct WordIterator<'a> {
    parser: Parser<'a>,
}

impl<'a> WordIterator<'a> {
    fn new(input: &'a str) -> Self {
        WordIterator {
            parser: Parser::new(input),
        }
    }
}

impl<'a> Iterator for WordIterator<'a> {
    type Item = &'a str;
    
    fn next(&mut self) -> Option<Self::Item> {
        self.parser.parse_word()
    }
}

// Configuration holder with references
#[derive(Debug)]
struct Config<'a> {
    name: &'a str,
    values: Vec<&'a str>,
}

impl<'a> Config<'a> {
    fn new(name: &'a str) -> Self {
        Config {
            name,
            values: Vec::new(),
        }
    }
    
    fn add_value(&mut self, value: &'a str) {
        self.values.push(value);
    }
    
    fn get_value(&self, index: usize) -> Option<&'a str> {
        self.values.get(index).copied()
    }
    
    fn all_values(&self) -> &[&'a str] {
        &self.values
    }
}

fn lifetime_data_structures_demo() {
    println!("\n=== Lifetime-Aware Data Structures Demo ===");
    
    // Parser example
    let text = "Hello world from Rust programming";
    let mut parser = Parser::new(text);
    
    println!("Parsing text: '{}'", text);
    println!("Words found:");
    
    let mut word_count = 0;
    while let Some(word) = parser.parse_word() {
        word_count += 1;
        println!("  {}: '{}'", word_count, word);
    }
    
    // Iterator example
    let sentence = "Rust lifetimes ensure memory safety";
    let word_iter = WordIterator::new(sentence);
    
    println!("\nUsing iterator on: '{}'", sentence);
    let words: Vec<&str> = word_iter.collect();
    println!("Collected words: {:?}", words);
    
    // Configuration example
    let config_data = vec!["localhost", "8080", "debug", "true"];
    let mut config = Config::new("server");
    
    for value in &config_data {
        config.add_value(value);
    }
    
    println!("\nConfiguration: {:?}", config);
    println!("First value: {:?}", config.get_value(0));
    println!("All values: {:?}", config.all_values());
    
    // Demonstrate zero-cost abstraction
    let large_text = "The quick brown fox jumps over the lazy dog. \
                     This sentence contains every letter of the alphabet. \
                     Rust's lifetime system ensures memory safety without \
                     runtime overhead or garbage collection.";
    
    let word_count = WordIterator::new(large_text).count();
    println!("\nTotal words in large text: {}", word_count);
    
    // Show that we can still access original data
    println!("Original text length: {} characters", large_text.len());
}

lifetime_data_structures_demo();

---

## 🎯 Guided Practice

### Exercise: Build a Text Analyzer with Lifetimes

Create a text analysis system that uses lifetimes effectively.

In [ ]:
// TODO: Complete the text analyzer with proper lifetime annotations

use std::collections::HashMap;

#[derive(Debug)]
struct TextAnalyzer<'a> {
    text: &'a str,
    word_cache: Option<Vec<&'a str>>,
}

impl<'a> TextAnalyzer<'a> {
    // TODO: Create a new analyzer
    fn new(text: &'a str) -> Self {
        TextAnalyzer {
            text,
            word_cache: None,
        }
    }
    
    // TODO: Get words, caching them for efficiency
    fn words(&mut self) -> &[&'a str] {
        if self.word_cache.is_none() {
            let words: Vec<&'a str> = self.text
                .split_whitespace()
                .map(|word| word.trim_matches(|c: char| !c.is_alphabetic()))
                .filter(|word| !word.is_empty())
                .collect();
            self.word_cache = Some(words);
        }
        self.word_cache.as_ref().unwrap()
    }
    
    // TODO: Count word frequencies
    fn word_frequencies(&mut self) -> HashMap<&'a str, usize> {
        let mut frequencies = HashMap::new();
        
        for word in self.words() {
            let word_lower = word.to_lowercase();
            // Note: We need to work with the original word reference
            // In a real implementation, you might want to normalize case
            *frequencies.entry(word).or_insert(0) += 1;
        }
        
        frequencies
    }
    
    // TODO: Find the most common word
    fn most_common_word(&mut self) -> Option<(&'a str, usize)> {
        self.word_frequencies()
            .into_iter()
            .max_by_key(|(_, count)| *count)
    }
    
    // TODO: Get words longer than specified length
    fn long_words(&mut self, min_length: usize) -> Vec<&'a str> {
        self.words()
            .iter()
            .filter(|word| word.len() >= min_length)
            .copied()
            .collect()
    }
    
    // TODO: Find words that start with a specific prefix
    fn words_starting_with(&mut self, prefix: &str) -> Vec<&'a str> {
        self.words()
            .iter()
            .filter(|word| word.to_lowercase().starts_with(&prefix.to_lowercase()))
            .copied()
            .collect()
    }
    
    // TODO: Get basic statistics
    fn statistics(&mut self) -> TextStatistics<'a> {
        let words = self.words();
        let word_count = words.len();
        let char_count = self.text.chars().count();
        let avg_word_length = if word_count > 0 {
            words.iter().map(|w| w.len()).sum::<usize>() as f64 / word_count as f64
        } else {
            0.0
        };
        
        let longest_word = words.iter().max_by_key(|w| w.len()).copied();
        let shortest_word = words.iter().min_by_key(|w| w.len()).copied();
        
        TextStatistics {
            word_count,
            char_count,
            avg_word_length,
            longest_word,
            shortest_word,
        }
    }
}

#[derive(Debug)]
struct TextStatistics<'a> {
    word_count: usize,
    char_count: usize,
    avg_word_length: f64,
    longest_word: Option<&'a str>,
    shortest_word: Option<&'a str>,
}

fn text_analyzer_demo() {
    println!("\n=== Text Analyzer Demo ===");
    
    let sample_text = "Rust is a systems programming language that runs blazingly fast, \
                      prevents segfaults, and guarantees thread safety. It accomplishes \
                      these goals by being memory safe without using garbage collection. \
                      Rust is fast and memory-efficient with no runtime or garbage collector.";
    
    let mut analyzer = TextAnalyzer::new(sample_text);
    
    // Basic statistics
    let stats = analyzer.statistics();
    println!("Text Statistics:");
    println!("  Words: {}", stats.word_count);
    println!("  Characters: {}", stats.char_count);
    println!("  Average word length: {:.2}", stats.avg_word_length);
    println!("  Longest word: {:?}", stats.longest_word);
    println!("  Shortest word: {:?}", stats.shortest_word);
    
    // Most common word
    if let Some((word, count)) = analyzer.most_common_word() {
        println!("\nMost common word: '{}' (appears {} times)", word, count);
    }
    
    // Long words
    let long_words = analyzer.long_words(8);
    println!("\nWords with 8+ characters: {:?}", long_words);
    
    // Words starting with 'g'
    let g_words = analyzer.words_starting_with("g");
    println!("\nWords starting with 'g': {:?}", g_words);
    
    // Word frequencies (top 5)
    let mut frequencies: Vec<_> = analyzer.word_frequencies().into_iter().collect();
    frequencies.sort_by(|a, b| b.1.cmp(&a.1));
    
    println!("\nTop 5 word frequencies:");
    for (word, count) in frequencies.iter().take(5) {
        println!("  '{}': {}", word, count);
    }
    
    println!("\n✅ Text analysis completed using zero-copy string slices!");
}

text_analyzer_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What are the three lifetime elision rules?
2. When do you need explicit lifetime annotations?
3. What does `'static` lifetime mean?
4. How do lifetime bounds work (e.g., `T: 'a`)?
5. What's the difference between `&'a str` and `&str`?
6. How do lifetimes help prevent dangling pointers?
7. Can a struct hold references without lifetime parameters?
8. What happens when lifetimes don't match in function calls?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How do lifetimes enable zero-cost abstractions in Rust?**
2. **What are the trade-offs between using lifetimes vs owned data?**
3. **How do lifetimes contribute to Rust's memory safety guarantees?**
4. **When might you choose to clone data instead of using lifetimes?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Advanced Traits & Trait Objects

In our next lesson, you'll learn about:
- Associated types and advanced trait patterns
- Trait objects and dynamic dispatch
- Higher-ranked trait bounds (HRTB)
- Blanket implementations and coherence rules
- Performance implications of different trait patterns

### How This Connects
Lifetimes are fundamental to understanding:
- How trait objects work with borrowed data
- Lifetime bounds in trait definitions
- Advanced iterator patterns and lazy evaluation
- Zero-cost abstractions in systems programming

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Write functions with explicit lifetime annotations?
- [ ] Design structs that hold references with proper lifetimes?
- [ ] Understand when lifetime elision applies?
- [ ] Resolve complex lifetime conflicts?
- [ ] Use lifetime bounds effectively?
- [ ] Build zero-cost abstractions using lifetimes?
- [ ] Explain how lifetimes prevent memory safety issues?

If you checked all boxes, excellent! You've mastered one of Rust's most advanced features.

---

**🎉 Advanced Achievement Unlocked!** You now understand how Rust achieves memory safety without garbage collection through its sophisticated lifetime system!